# Details of tiktoken, Explained

People who have used OpenAI's GPT series services through API calls are definitely familiar with `tiktoken`. Before making the API call, we can use `tiktoken` to calculate the expected number of tokens we will consume, which helps us estimate the cost (or avoid exceeding the input length limit of the GPT series models).
Most people only use `tiktoken` to calculate the number of tokens at a basic level, as shown in the following simple usage:

In [3]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4")
text = "hello world"
number_of_tokens = len(enc.encode(text))

This article provides an introduction to tiktoken, starting from the basics and gradually delving into some of its details.

## There are only five types of Tokenizers in tiktoken.

Most people use the function `enc = tiktoken.encoding_for_model("gpt-4")` to find the corresponding tokenizer based on the model name. Currently, OpenAI has over 30 models available, including commonly used ones such as text-davinci-003, gpt-3.5-turbo, gpt-4, and so on.

However, by examining the `model.py` file in `tiktoken`, we can see that there are actually only five types of tokenizers: `cl100k_base`, `p50k_base`, `r50k_base`, `p50k_edit`, and `gpt2`. The specific mapping table can be seen in the code below. The parameter used in `encoding_for_model` is first used for exact matching, and if not found, it is converted to a prefix matching. The familiar gpt-4 series models we know actually use `cl100k_base`, which has approximately `100,000` tokens. The earlier text-davinci-003 (gpt-3 series) uses `p50k_base`/`r50k_base`, which have around `50,000` tokens in their vocabulary.

```python
MODEL_PREFIX_TO_ENCODING: dict[str, str] = {
    # chat
    "gpt-4-": "cl100k_base",  # e.g., gpt-4-0314, etc., plus gpt-4-32k
    "gpt-3.5-turbo-": "cl100k_base",  # e.g, gpt-3.5-turbo-0301, -0401, etc.
}

MODEL_TO_ENCODING: dict[str, str] = {
    # chat
    "gpt-4": "cl100k_base",
    "gpt-3.5-turbo": "cl100k_base",
    # text
    "text-davinci-003": "p50k_base",
    "text-davinci-002": "p50k_base",
    "text-davinci-001": "r50k_base",
    "text-curie-001": "r50k_base",
    "text-babbage-001": "r50k_base",
    "text-ada-001": "r50k_base",
    "davinci": "r50k_base",
    "curie": "r50k_base",
    "babbage": "r50k_base",
    "ada": "r50k_base",
    # code
    "code-davinci-002": "p50k_base",
    "code-davinci-001": "p50k_base",
    "code-cushman-002": "p50k_base",
    "code-cushman-001": "p50k_base",
    "davinci-codex": "p50k_base",
    "cushman-codex": "p50k_base",
    # edit
    "text-davinci-edit-001": "p50k_edit",
    "code-davinci-edit-001": "p50k_edit",
    # embeddings
    "text-embedding-ada-002": "cl100k_base",
    # old embeddings
    "text-similarity-davinci-001": "r50k_base",
    "text-similarity-curie-001": "r50k_base",
    "text-similarity-babbage-001": "r50k_base",
    "text-similarity-ada-001": "r50k_base",
    "text-search-davinci-doc-001": "r50k_base",
    "text-search-curie-doc-001": "r50k_base",
    "text-search-babbage-doc-001": "r50k_base",
    "text-search-ada-doc-001": "r50k_base",
    "code-search-babbage-code-001": "r50k_base",
    "code-search-ada-code-001": "r50k_base",
    # open source
    "gpt2": "gpt2",
}
```

So, if you know the name of the Tokenizer, you can directly construct it, for example: `enc = tiktoken.get_encoding("cl100k_base")`.

## Attributes of Tokenizer and Hidden Tokens

The `Tokenizer` object has several hidden attributes: (According to convention, the variable name for the `Tokenizer` is `enc`.)


In [4]:
print(enc.special_tokens_set) # all the special token
tokens = enc.token_byte_values() # all the vocabulary (excluding special tokens)
print(enc.n_vocab) #size of vocabulary

{'<|fim_middle|>', '<|endofprompt|>', '<|fim_prefix|>', '<|fim_suffix|>', '<|endoftext|>'}
100277


Generally speaking, there are only two types of tokens: regular tokens and special tokens. Therefore, in general, the size of `n_vocab` is the sum of regular tokens and special tokens. However, through experimentation, I found that `cl100k_base` is an exception. It has `100,277` tokens (i.e., the `n_vocab` attribute), but only `100,256` regular tokens and `5` special tokens. In other words, OpenAI has hidden at least `16` special tokens.

## Model File of the Tokenizer

How are hidden tokens discovered? Here we can see the following code snippet in the `tiktoken_ext/openai_public.py` file:
```python
def cl100k_base():
    mergeable_ranks = load_tiktoken_bpe(
        "https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken"
    )
    special_tokens = {
        ENDOFTEXT: 100257,
        FIM_PREFIX: 100258,
        FIM_MIDDLE: 100259,
        FIM_SUFFIX: 100260,
        ENDOFPROMPT: 100276,
    }
    return {
        "name": "cl100k_base",
        "pat_str": r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+""",
        "mergeable_ranks": mergeable_ranks,
        "special_tokens": special_tokens,
    }
```

In other words, the model file for the `cl100k_base` tokenizer is stored at this URL (this indicates that the tiktoken package does not include the tokenizer model file and it will be downloaded when called, which can get stuck if the network environment is poor). Based on this, five special tokens are added.

The specific content of this file format (with the `.tiktoken` extension) is the base64 encoding of the character sequence of tokens. Please refer to these two functions in `tiktoken/load.py` for more information:

```python
def dump_tiktoken_bpe(bpe_ranks: dict[bytes, int], tiktoken_bpe_file: str) -> None:
    try:
        import blobfile
    except ImportError as e:
        raise ImportError(
            "blobfile is not installed. Please install it by running `pip install blobfile`."
        ) from e
    with blobfile.BlobFile(tiktoken_bpe_file, "wb") as f:
        for token, rank in sorted(bpe_ranks.items(), key=lambda x: x[1]):
            f.write(base64.b64encode(token) + b" " + str(rank).encode() + b"\n")


def load_tiktoken_bpe(tiktoken_bpe_file: str) -> dict[bytes, int]:
    # NB: do not add caching to this function
    contents = read_file_cached(tiktoken_bpe_file)
    return {
        base64.b64decode(token): int(rank)
        for token, rank in (line.split() for line in contents.splitlines() if line)
    }
```

There is a problem here: the indices of special tokens (for the `cl100k_base` tokenizer) are not consecutive. It jumps from 100260 directly to 100276, hiding 16 special tokens in between.

The official documentation also validates our idea, as mentioned in its README, that the Tokenizer can be extended in this way:

```python
cl100k_base = tiktoken.get_encoding("cl100k_base")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
enc = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        "<|im_start|>": 100264,
        "<|im_end|>": 100265,
    }
)
```

This should be a specific Tokenizer used internally by OpenAI, from which we can deduce the following: Token with index 100264 is used to mark the boundary of user input during the ChatGPT dialogue, while token with index 100265 is used to mark the boundary of machine input (refer to the [ChatML](https://github.com/openai/openai-python/blob/main/chatml.md) format for more details).

As for the remaining 14 special tokens, we are not aware of their specific purposes. However, based on our speculation, some of them might be related to image inputs (such as marking the beginning and end of an image), while others could be associated with the dialogue roles, such as the user/assistant/system roles in the ChatGPT API.

## Protection of special tokens (to prevent injection attacks)

The aforementioned special tokens are typically used to trigger specific capabilities such as image comprehension and model instructions, so user input needs to be prohibited. This is specifically implemented within the encode function.

```python
    def encode(
        self,
        text: str,
        *,
        allowed_special: Union[Literal["all"], AbstractSet[str]] = set(), 
        disallowed_special: Union[Literal["all"], Collection[str]] = "all",
    ) -> list[int]:
```

For each special token, there are three possible processing methods:

- Trigger a specific capability of the model as a special token.
- Parse it as regular text.
- Throw an error.

`allowed_special` refers to the special tokens that are allowed to be parsed, while `disallowed_special` refers to the tokens that will result in an error. By default, special tokens are not allowed to be parsed, and they will all result in an error.

In [ ]:
enc.encode('<|endoftext|>') # error

However, when we attempted to use the Tokenizer provided by the OpenAI website, this sentence was parsed as regular text.

![Image](https://github-production-user-asset-6210df.s3.amazonaws.com/23236638/242950647-6c8a231a-b3d8-4fa9-b363-fdd597dfe51e.png)

Therefore, for text inputs in user calls, OpenAI should be using the parsing method `enc.encode(text, disallowed_special=set())`.
As a result, it is not possible for users to insert special tokens in OpenAI API calls.

## How to deal with UNK/OOV

A few years ago, when NLP development was not as mature, Tokenizers primarily relied on dictionaries, often resulting in out-of-vocabulary (OOV) situations during the Tokenization phase, where the token could only be marked as an unknown word (UNK).

Now, OpenAI's `tiktoken`, based on the byte-pair encoding (BPE) tokenization method, completely avoids the OOV/UNK problem. This is because OpenAI's tiktoken treats the input sequence as a byte sequence and retains all single-byte tokens in its vocabulary, allowing it to handle worst-case scenarios at the character level.

The BPE tokenization method essentially involves prefix matching during the tokenization phase, allowing us to construct this prefix tree:

In [8]:
from collections import defaultdict

def build_tree(lst, enc):
    tree = defaultdict(dict)
    for item in lst:
        current = tree
        for byte in item:
            if byte not in current:
                current[byte] = {}
            current = current[byte]
        current['end'] = enc._mergeable_ranks[item]
    return tree

tree = build_tree(enc.token_byte_values(), enc)

This tree has a corresponding token for each byte (within the range of 0-256).

In [13]:
for i in range(256):
    assert 'end' in tree[i]

Therefore, by using BPE and incorporating all characters into the vocabulary, we will no longer encounter OOV/UNK issues.

## Preprocessing of Sentences by tiktoken

Those familiar with BERT know that BERT adds a `[CLS]` token at the beginning of a sentence and also marks the start and end of each word (e.g., `##est` indicates the last three characters of a word). However, OpenAI's tiktoken does not perform these preprocessing steps.

Whether a character appears at the beginning or end of a word does not affect the token, as shown in the below:

In [14]:
print(enc.encode('unun')) # [359, 359]
print(enc.encode('un')) # [359]

[359, 359]
[359]


We can see that the initial "un" and the final "un" are both the same token, 359. Additionally, there are no special tokens like `[CLS]` added at the beginning.

## About the pat_str

In each Tokenizer definition within `tiktoken`, we can see a string of peculiar `pat_str`. Its full name is "pattern string," which is a regular expression.

```python
def cl100k_base():
    # omitted
    return {
        "name": "cl100k_base",
        "pat_str": r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+""",
        "mergeable_ranks": mergeable_ranks,
        "special_tokens": special_tokens,
    }
```

The core logic of `tiktoken` is written in Rust. The core code for its encoding is as follows:
```rust
    fn _encode_ordinary_native(&self, text: &str) -> Vec<usize> {
        // This is the core of the encoding logic; the other functions in here
        // just make things complicated :-)
        let regex = self._get_tl_regex();
        let mut ret = vec![];
        for mat in regex.find_iter(text) {
            let piece = mat.unwrap().as_str().as_bytes();
            if let Some(token) = self.encoder.get(piece) {
                ret.push(*token);
                continue;
            }
            ret.extend(&byte_pair_encode(piece, &self.encoder));
        }
        ret
    }
```

Among them, the `regex` variable is derived from the compilation of `pat_str`. Therefore, we have a rough idea of the core logic of `tiktoken`: segmenting the input based on `pat_str` and then tokenizing each segment. Perhaps this is more accurate than simply dividing by spaces?

I couldn't find any documentation on how this `pat_str` is constructed. The regular expression is long and difficult to understand. The documentation for the `find_iter` function in Rust can be found at [Regex in fancy_regex](https://docs.rs/fancy-regex/latest/fancy_regex/struct.Regex.html#method.find_iter). Interested friends can take a look.

I tried writing a Rust code to test:

```rust
use fancy_regex::Regex;

fn main() {
    let re = Regex::new(r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+").unwrap();
    let s = "This is a sentence about hello world";
    for mat in re.find_iter(s) {
        println!("Matched: {}", mat.unwrap().as_str());
    }
}
```

Results are:
```
Matched: This
Matched:  is
Matched:  a
Matched:  sentence
Matched:  about
Matched:  hello
Matched:  world
```

As seen, the input sentence is segmented into several smaller segments, which are then tokenized using BPE.

## Regarding multithreading and parallelism

The tokenization process is generally sequential, where the calculation of subsequent tokens depends on the results of the previous ones.

However, on the `tiktoken` GitHub repository, there is a figure indicating a result achieved through multithreading and parallel processing:

![image](https://github-production-user-asset-6210df.s3.amazonaws.com/23236638/242954855-ef6af119-f812-437e-b64d-8bed119e5fef.png)

After carefully reading the code, I found that the characteristics of multithreading only appear in the `encode_batch` function. In other words, the multiprocessing feature is only enabled when tokenizing multiple sentences at once.

Here are the timing results I obtained in IPython. It can be observed that when tokenizing less than 8 sentences simultaneously, the time taken is roughly equivalent to tokenizing a single sentence. However, when tokenizing 16 sentences simultaneously, the time taken is approximately twice as long. This is because the default setting uses 8 threads.

In [15]:
test = 'nonsense' * 1000

In [22]:
%timeit -n 10 out = enc.encode(test)

23.7 ms ± 832 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit -n 10 out = enc.encode_batch([test, test])

24.6 ms ± 888 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit -n 10 out = enc.encode_batch([test] * 8)

27.4 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit -n 10 out = enc.encode_batch([test] * 16)

52.4 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The specific parallel details involve the `pyo3` package in the Rust language, allowing multi-threading parallelism through `py.allow_threads`. For more information, please refer to [Parallelism](https://pyo3.rs/v0.13.2/parallelism).

## Further reading

For a detailed explanation of the specific principles behind byte-pair encoding and an introduction to various tokenization techniques, please refer to the following blog posts:

- https://www.freecodecamp.org/news/evolution-of-tokenization/
- https://www.freecodecamp.org/news/train-algorithms-from-scratch-with-hugging-face/
- https://zhuanlan.zhihu.com/p/631463712 (chinese)
- https://zhuanlan.zhihu.com/p/86965595 (chinese)